In [1]:
import urllib.request

urllib.request.urlretrieve('https://archive.ics.uci.edu/static/public/53/iris.zip', 'data.zip')

('data.zip', <http.client.HTTPMessage at 0x7f24cb1ff220>)

In [2]:
!mkdir data
!unzip data.zip -d data/

Archive:  data.zip
  inflating: data/Index              
  inflating: data/bezdekIris.data    
  inflating: data/iris.data          
  inflating: data/iris.names         


In [3]:
import pandas as pd

#read data
data = pd.read_csv('data/iris.data', header=None)

data[4] = data[4].replace('Iris-setosa', 0)
data[4] = data[4].replace('Iris-virginica', 1)
data[4] = data[4].replace('Iris-versicolor', 2)

#shuffle

data = data.sample(frac=1).reset_index(drop=True)

#change label column index
data = data[[4, 0, 1, 2, 3]]

#split
train_data = data[:120]
val_data = data[120:]

In [4]:
import boto3

bucket_name = 'sagemaker-build-deploy'

train_data.to_csv('data.csv', header=False, index=False)
key = 'data/train/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

val_data.to_csv('data.csv', header=False, index=False)

key = 'data/val/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

In [7]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

key = 'model/xgb_model'
s3_output_location = 's3://{}/{}'.format(bucket_name, key)

xgb_model = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, 'xgboost'),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    train_volume_size=5,
    output_path=s3_output_location,
    sagemaker_Session = sagemaker.Session()
    
)

xgb_model.set_hyperparameters(max_depth=5,
                             eta=0.2,
                             gamma=4,
                             min_child_weight=6,
                             silent=0,
                             objective='multi:softmax',
                             num_class=3,
                             num_round=10 
                             )

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [8]:
train_data = 's3://{}/{}'.format(bucket_name, 'data/train')
val_data = 's3://{}/{}'.format(bucket_name, 'data/val')

train_channel = sagemaker.session.s3_input(train_data, content_type='text/csv')
val_channel = sagemaker.session.s3_input(val_data, content_type='text/csv')


data_channels = {'train': train_channel, 'validation': val_channel}

xgb_model.fit(inputs=data_channels)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: xgboost-2023-11-01-02-53-02-290


2023-11-01 02:53:02 Starting - Starting the training job...
2023-11-01 02:53:27 Starting - Preparing the instances for training......
2023-11-01 02:54:22 Downloading - Downloading input data...
2023-11-01 02:54:58 Training - Downloading the training image...
2023-11-01 02:55:28 Uploading - Uploading generated training modelArguments: train
[2023-11-01:02:55:21:INFO] Running standalone xgboost training.
[2023-11-01:02:55:21:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8235.73mb
[2023-11-01:02:55:21:INFO] Determined delimiter of CSV input is ','
[02:55:21] S3DistributionType set as FullyReplicated
[02:55:21] 120x4 matrix with 480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-11-01:02:55:21:INFO] Determined delimiter of CSV input is ','
[02:55:21] S3DistributionType set as FullyReplicated
[02:55:21] 30x4 matrix with 120 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delim

In [9]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-11-01-02-58-40-494
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-11-01-02-58-40-494
INFO:sagemaker:Creating endpoint with name xgboost-2023-11-01-02-58-40-494


---!